In [44]:
import pandas as pd
import pickle

In [45]:
with open("./test/sales_amount_df.pickle", mode='rb') as f:
    #with open("./src/test/sales_amount_df.pickle", mode='rb') as f:
        sales_amount_df = pickle.load(f)
    
with open("./test/init_stock_df.pickle", mode='rb') as f:
    init_stock_df = pickle.load(f)

In [ ]:
#TODO

#期首在庫と販売の齟齬がないようにしないとダメ。
#→今販売量からやってしまったが。。。。


In [59]:
init_stock_df

,通称,期首在庫
22,3PR-1,131.0
23,3PR-1SW,244.0
24,3XR-1,0.0
25,3XR-1SW,3775.0
26,3XR-1SWD,332.0
27,3XR-1UWS,864.0
28,3XR-1UWD,229.0


In [47]:

params_folder_path = "./順序シミュ用パラメータ/ver0.0.2_杉浦作成_2024年度パラメータ_20240208"    #大元のパラメータファイル
params_folder_path = "./順序シミュ用パラメータ/ver0.0.2_杉浦作成_2024年度パラメータ_20240208"    #大元のパラメータファイル
init_stock_file_path = params_folder_path + "/期首在庫.xlsx"
sales_file_path = params_folder_path + "/販売計画.xlsx"
quality_ratio_path = params_folder_path + "/品質収率.xlsx"
basic_info_path = params_folder_path + "/製品基本データ.xlsx"  
name_path = params_folder_path+"/名称対応表.xlsx"

In [48]:
plant = "L6"

In [49]:
#L1だけでなくほかの工場も必要

init_stock_df_org = pd.read_excel(init_stock_file_path,sheet_name=plant)
sales_file_df = pd.read_excel(sales_file_path,sheet_name=plant)
quality_ratio_df = pd.read_excel(quality_ratio_path,sheet_name=plant)
basic_info_df = pd.read_excel(basic_info_path,sheet_name=plant)
name_df = pd.read_excel(name_path,sheet_name=plant)

In [50]:
#前処理
quality_ratio_df = quality_ratio_df.drop(columns=["巻長・巻き方"])
init_stock_df_org = init_stock_df_org.rename(columns={'優先品質': '品質'})
basic_info_df = basic_info_df.rename(columns={"巻長・巻き方":"通称"})
sales_amount_df = sales_amount_df[sales_amount_df["工場"] == plant].drop(columns=["工場"]).rename(columns={"品種名":"通称"})
init_stock_df = init_stock_df[init_stock_df["工場"] == plant].drop(columns=["工場"]).rename(columns={"品種名":"通称"})
init_stock_df

,通称,期首在庫
22,3PR-1,131.0
23,3PR-1SW,244.0
24,3XR-1,0.0
25,3XR-1SW,3775.0
26,3XR-1SWD,332.0
27,3XR-1UWS,864.0
28,3XR-1UWD,229.0


In [58]:
# 左結合
left_join_df = pd.merge(name_df, init_stock_df_org, on='品質', how='left')
left_join_df = pd.merge(left_join_df, quality_ratio_df, on='品質', how='left')
left_join_df = pd.merge(left_join_df, basic_info_df,on="通称",how="left")
left_join_df_filtered = left_join_df[left_join_df['収率'] != 0]
left_join_df_filtered

,工場,通称,7桁製品名,10桁製品名,巻長・巻き方,品質,期首在庫,収率,定尺,余尺,試料長,幅
0,L6,3PR-1,KC3PR-1 1330X1M,KC3PR-1 1330(L6),KC3PR-1 1330(L6)_3900_オシ,3PR-W/L6_標準品質,32,100,3900,20,15,1.33
2,L6,3PR-1SW,KC3PR-1SW 1490X1M,KC3PR-1SW 1490(L6),KC3PR-1SW 1490(L6)_3900_オシ,3PR-SW/L6_標準品質,60,100,3900,20,15,1.49
5,L6,3PR-1,KC3PR-1 1330X1M,KC3PR-1 1330(L6),KC3PR-1 1330(L6)_3930_オシ,3PR-W_3930/L6_標準品質,0,100,3900,20,15,1.33
7,L6,3PR-1SW,KC3PR-1SW 1490X1M,KC3PR-1SW 1490(L6),KC3PR-1SW 1490(L6)_3930_オシ,3PR-SW_3930/L6_標準品質,0,100,3900,20,15,1.49
10,L6,3XR-1,KC3XR-1 1330X1M,KC3XR-1 1330(L6),KC3XR-1 1330X1M(L6)_3930_オシ,3XR-W_3930/L6_標準品質,0,100,3900,20,15,1.33
14,L6,3XR-1,KC3XR-1 1330X1M,KC3XR-1 1330(L6),KC3XR-1 1330(L6)_3900_オシ,3XR-W_3900/L6_標準品質,0,100,3900,20,15,1.33
18,L6,3XR-1,KC3XR-1 1330X1M,KC3XR-1 1330(L6),KC3XR-1 1330(L6)_6000_オシ,3XR-W_6000/L6_標準品質,0,100,3900,20,15,1.33
20,L6,3XR-1SW,KC3XR-1SW 1490X1M,KC3XR-1SW 1490(L6),KC3XR-1SW 1490(L6)_3900_オシ,3XR-SW_3900/L6_標準品質,928,100,3900,20,15,1.49
24,L6,3XR-1SW,KC3XR-1SW 1490X1M,KC3XR-1SW 1490(L6),KC3XR-1SW 1490X1M(L6)_3930_オシ,3XR-SW_3930/L6_標準品質,0,100,3900,20,15,1.49
28,L6,3XR-1SWD,KC3XR-1SWD 1540X1M,KC3XR-1SWD 1540(L6),KC3XR-1SWD 1540(L6)_3900_オシ,3XR-SWD_3900/L6_標準品質,79,100,3900,20,15,1.54


In [52]:
value_counts = left_join_df_filtered['通称'].value_counts()
duplicate_name_list = value_counts[value_counts > 1].index.tolist()

#重複があるものについては、期首在庫が0よりおおきいものを採用
for name in duplicate_name_list:
    left_join_df_filtered = left_join_df_filtered[~((left_join_df_filtered['通称'] == name) & (left_join_df_filtered['期首在庫'] == 0))]

#それでも重複がある場合はどちらかだけにする
left_join_df_filtered = left_join_df_filtered.drop_duplicates(subset=['通称'])


In [53]:
#必要なデータがすべてそろったデータフレーム

df_all = pd.merge(left_join_df_filtered, sales_amount_df,on="通称",how="left").fillna(0)
df_all

,工場,通称,7桁製品名,10桁製品名,巻長・巻き方,品質,期首在庫,収率,定尺,余尺,...,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月
0,L6,3PR-1,KC3PR-1 1330X1M,KC3PR-1 1330(L6),KC3PR-1 1330(L6)_3900_オシ,3PR-W/L6_標準品質,32,100,3900,20,...,118.00000,118.00000,118.00000,118.00000,118.00000,118.00000,118.00000,0.00000,0.00000,0.00000
1,L6,3PR-1SW,KC3PR-1SW 1490X1M,KC3PR-1SW 1490(L6),KC3PR-1SW 1490(L6)_3900_オシ,3PR-SW/L6_標準品質,60,100,3900,20,...,132.00000,132.00000,132.00000,132.00000,132.00000,132.00000,132.00000,0.00000,0.00000,0.00000
2,L6,3XR-1SW,KC3XR-1SW 1490X1M,KC3XR-1SW 1490(L6),KC3XR-1SW 1490(L6)_3900_オシ,3XR-SW_3900/L6_標準品質,928,100,3900,20,...,2145.53730,2539.62420,2539.62420,2539.62420,2636.24730,2636.24730,2636.24730,2610.93120,2610.93120,2610.93120
3,L6,3XR-1SWD,KC3XR-1SWD 1540X1M,KC3XR-1SWD 1540(L6),KC3XR-1SWD 1540(L6)_3900_オシ,3XR-SWD_3900/L6_標準品質,79,100,3900,20,...,260.78389,255.11467,255.11467,255.11467,266.45310,266.45310,266.45310,249.44546,249.44546,249.44546
4,L6,3XR-1UWD,KC3XR-1UWD 1980X1M,KC3XR-1UWD 1980(L6),KC3XR-1UWD 1980(L6)_3900_オシ,3XR-UWD(6in)/L6_標準品質,0,100,3900,20,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,229.00000
5,L6,3XR-1UWS,KC3XR-1UWS 1720X1M,KC3XR-1UWS 1720(L6),KC3XR-1UWS 1720(L6)_3900_オシ,3XR-UWS/L6_標準品質,184,100,3900,20,...,316.59252,696.50355,696.50355,696.50355,759.82206,753.49021,759.82206,734.68344,734.68344,734.68344


In [54]:
#本数に直す
#基本予算ではない場合、month_listが変わってくる。注意

month_list = ["4月","5月","6月","7月","8月","9月","10月","11月","12月","1月","2月","3月"]

for month in month_list:

    df_all[month] = df_all[month]*1000/(df_all["定尺"]*df_all["幅"])

df_all


,工場,通称,7桁製品名,10桁製品名,巻長・巻き方,品質,期首在庫,収率,定尺,余尺,...,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月
0,L6,3PR-1,KC3PR-1 1330X1M,KC3PR-1 1330(L6),KC3PR-1 1330(L6)_3900_オシ,3PR-W/L6_標準品質,32,100,3900,20,...,22.749181,22.749181,22.749181,22.749181,22.749181,22.749181,22.749181,0.000000,0.000000,0.000000
1,L6,3PR-1SW,KC3PR-1SW 1490X1M,KC3PR-1SW 1490(L6),KC3PR-1SW 1490(L6)_3900_オシ,3PR-SW/L6_標準品質,60,100,3900,20,...,22.715539,22.715539,22.715539,22.715539,22.715539,22.715539,22.715539,0.000000,0.000000,0.000000
2,L6,3XR-1SW,KC3XR-1SW 1490X1M,KC3XR-1SW 1490(L6),KC3XR-1SW 1490(L6)_3900_オシ,3XR-SW_3900/L6_標準品質,928,100,3900,20,...,369.219979,437.037377,437.037377,437.037377,453.664997,453.664997,453.664997,449.308415,449.308415,449.308415
3,L6,3XR-1SWD,KC3XR-1SWD 1540X1M,KC3XR-1SWD 1540(L6),KC3XR-1SWD 1540(L6)_3900_オシ,3XR-SWD_3900/L6_標準品質,79,100,3900,20,...,43.420561,42.476635,42.476635,42.476635,44.364486,44.364486,44.364486,41.532711,41.532711,41.532711
4,L6,3XR-1UWD,KC3XR-1UWD 1980X1M,KC3XR-1UWD 1980(L6),KC3XR-1UWD 1980(L6)_3900_オシ,3XR-UWD(6in)/L6_標準品質,0,100,3900,20,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.655530
5,L6,3XR-1UWS,KC3XR-1UWS 1720X1M,KC3XR-1UWS 1720(L6),KC3XR-1UWS 1720(L6)_3900_オシ,3XR-UWS/L6_標準品質,184,100,3900,20,...,47.196261,103.831775,103.831775,103.831775,113.271029,112.327103,113.271029,109.523470,109.523470,109.523470


In [55]:
sales_file_df2 = pd.DataFrame(columns=sales_file_df.columns)
sales_file_df2["優先品質"] = sales_file_df["優先品質"]
sales_file_df2 = sales_file_df2.fillna(0)

C:\Users\e13753\AppData\Local\Temp\ipykernel_25580\3731912451.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sales_file_df2 = sales_file_df2.fillna(0)


In [56]:
# df2のIDに基づいてdf1を上書き
for index, row in df_all.iterrows():
    df1_index = sales_file_df2[sales_file_df2['優先品質'] == row['品質']].index
    for col in month_list:  # IDカラムを除外
        sales_file_df2.loc[df1_index, col] = row[col]

C:\Users\e13753\AppData\Local\Temp\ipykernel_25580\2309479951.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '22.749180643917487' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sales_file_df2.loc[df1_index, col] = row[col]
C:\Users\e13753\AppData\Local\Temp\ipykernel_25580\2309479951.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '22.749180643917487' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sales_file_df2.loc[df1_index, col] = row[col]
C:\Users\e13753\AppData\Local\Temp\ipykernel_25580\2309479951.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '22.749180643917487' has dtype incompatible with int64, please explicitly cast to a compatible dtype

In [57]:
sales_file_df2

,優先品質,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月
0,3PR-W/L6_標準品質,22.749181,22.749181,22.749181,22.749181,22.749181,22.749181,22.749181,22.749181,22.749181,0.000000,0.000000,0.000000
1,3PR-W/L6_滞留品,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3PR-SW/L6_標準品質,22.715539,22.715539,22.715539,22.715539,22.715539,22.715539,22.715539,22.715539,22.715539,0.000000,0.000000,0.000000
3,3PR-SW/L6_限定品質,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3PR-SW/L6_滞留品,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,3PR-W_3930/L6_標準品質,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,3PR-W_3930/L6_滞留品,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,3PR-SW_3930/L6_標準品質,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,3PR-SW_3930/L6_限定品質,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,3PR-SW_3930/L6_滞留品,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
